# Pandas
Data structures in pandas
- `Series` objects: 1D array, similar to a column in a spreadsheet
- `DataFrame` objects: 2D table, similar to a spreadsheet
- `Panel` objects: Dictionary of DataFrames, similar to sheet in MS Excel

## Series

In [ ]:
import pandas as pd
import numpy as np

birthyear = pd.Series([1984, 1985, 1992])
print(birthyear)
print(birthyear.index)
print()

weight = pd.Series([68, 83, 112],index=["alice", "bob", "charles"])
print(weight)
print(weight.index)

## DataFrame Creation
- from dict
- from iterables (with column names)
- from ndarray (with colum names)

In [ ]:
weight = pd.Series([68, 83, 112],index=["alice", "bob", "charles"])
birthyear = pd.Series([1984, 1985, 1992], index=["bob", "alice", "charles"], name="year")
children = pd.Series([0, 3], index=["charles", "bob"])
hobby = pd.Series(["Biking", "Dancing"], index=["alice", "bob"])

people_dict = { "weight": weight,
                "birthyear": birthyear,
                "children": children,
                "hobby": hobby}

people = pd.DataFrame(people_dict)
print(people)

In [ ]:
print('shape:', people.shape)
print(people.dtypes)
print('index:', people.index)
print('columns:', people.columns)

In [ ]:
people['birthyear']     # extracting column --> Series

In [ ]:
people['birthyear'] < 1990      # like numpy boolean array

In [ ]:
old_people = people[people['birthyear'] < 1990]    # like numpy boolean array indexing
old_people

In [ ]:
people_abbr = people[['birthyear', 'weight']]   # like numpy integer array indexing
people_abbr

In [ ]:
people['weight'].mean()

## Data Preparation
Can import/export data from/to csv, excel, json, and text
### Importing data
```Python
df = pd.read_csv("diabetes.csv")
df = pd.read_csv("diabetes.txt", sep="\s")
df = pd.read_excel('diabetes.xlsx')
df = pd.read_excel('diabetes_multi.xlsx', sheet_name=1) # extract 2nd sheet
df = pd.read_json("diabetes.json")
```
### Exporting DataFrame
```Python
df.to_csv("diabetes_out.csv", index=False)
df.to_csv('diabetes_out.txt', header=df.columns, index=None, sep=' ')
df.to_excel("diabetes_out.xlsx", index=False)
df.to_json("diabetes_out.json")
```

In [ ]:
df = pd.read_csv("diabetes.csv")
print(df.info())
df.head()

In [ ]:
print(list(df.columns))
df.columns 

In [ ]:
df.index

#### Fetching rows
- `.loc[]`: location
- `iloc[]`: iteger location (position) 

In [ ]:
df.loc[1]

In [ ]:
df.loc[[1, 2, 767], ['Pregnancies', 'Outcome']]

In [ ]:
df.iloc[10:13, :3]    # row and column position

#### Conditional slicing

In [ ]:
df[df.BloodPressure > 110] 

In [ ]:
df.loc[df['BloodPressure'] > 110, ['Pregnancies', 'Glucose', 'BloodPressure']]

In [ ]:
df.describe()

### Dealing with missing data
#### Checking for missing values
값이 없거나 비정상적인 것이 없는지 체크하고 있다면 drop하거나 적절한 값을 채워 넣아야 한다.

예를 들어 값이 없는 경우를 만들어 보고 이를 보완해 보자.

In [ ]:
df2 = df.copy()   # make a copy
df2.loc[2:5, 'DiabetesPedigreeFunction'] = None
df2.loc[[4, 6], 'Glucose'] = None
df2.head(7)

In [ ]:
df2.isnull().sum()    # NaN count

#### 1) Dropping missing values

In [ ]:
df3 = df2.copy()
df3 = df3.dropna()      # drop rows containing missing values
# df3.dropna(inplace=True)  # do not create new DataFrame
# df3.dropna(inplace=True, axis=1) # drop columns containing missing values
# df3.dropna(inplace=True, how='all')   # drop both rows and columns
df3.shape

#### 2) Replacing missing values
- with average
- with moving average (rolling mean): time series 등의 경우

In [ ]:
df3 = df2.copy()
avg = df3.DiabetesPedigreeFunction.mean()
df3.DiabetesPedigreeFunction.fillna(avg, inplace=True)
avg1 = df3['Glucose'].mean()
df3['Glucose'].fillna(avg1, inplace=True)
df3.head(7)

### Dealing with Duplicate Data

In [ ]:
df4 = pd.concat([df, df3.loc[:10]])
print(df4.shape)
df4[df4.duplicated()]

In [ ]:
df4.drop_duplicates(inplace=True)
df4.shape

#### Renaming columns

In [ ]:
df3.rename(columns = {'DiabetesPedigreeFunction':'DPF'}, inplace = True)
df3.head()

In [ ]:
df3.columns = ['임신기간', 'Glucose', '혈압', 'SkinThickness', 'Insulin', 'BMI', 'DPF', '나이', 'Outcome']
print(df3.columns)
df3.혈압.head()

## Data Anaysis
`.mean(), .max(), .min(), .sum(), .mode(), .value_counts()` methods

In [ ]:
df3['Outcome'].value_counts()

#### Aggregating data

In [ ]:
df3.groupby('Outcome').mean()

In [ ]:
df3.groupby(['임신기간', 'Outcome']).mean()

#### Pivoting

In [ ]:
pd.pivot_table(df3, values="BMI", index='임신기간', columns=['Outcome'], aggfunc=np.mean)

#### Correlation

In [ ]:
corr = df3.corr()
corr

In [ ]:
df3[['Glucose', '혈압', 'BMI']].plot(subplots=True, figsize=(10,10))

In [ ]:
df3['혈압'].hist(bins=50)

In [ ]:
df3.plot.scatter(x='BMI', y='혈압')

### Iterate over rows

In [ ]:
for index, row in df3.iterrows():
    status = 'Yes' if row['Outcome'] else 'No'
    print(index, status) 

### `apply()` method
used to apply a function along with an axis of the DataFrame


In [ ]:
df3['Status'] = df3['Outcome'].apply(lambda x: ('No', 'Yes')[x])
df3.head()

In [ ]:
def classify(bmi):
    if bmi < 18.5: return '저체중'
    if bmi < 23.0: return '정상'
    if bmi < 25.0: return '과체중'
    return '비만'

df3['비만도'] = df3['BMI'].apply(classify)
df3.head()

In [ ]:
df3['비만도'].value_counts()

### sorting
sort by columns

In [ ]:
df3.sort_values('혈압', ascending=False, inplace=True, na_position='first')
df3.head()

sort by index

In [ ]:
df3.sort_index(inplace=True)
df3.head()

### `concat`, `merge`, `join` DataFrames
> 참고: https://yganalyst.github.io/data_handling/Pd_12/

### Reshaping DataFrames: `stack`, `unstack`
Multi-level columns

In [ ]:
multi_col = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Temperature', 'C'), ('Temperature', 'F')]
)
print(multi_col)

df_multi = pd.DataFrame(
    [[13, 8, 0], [19, 6, 0]],
    index=['London', 'Oxford'],
    columns=multi_col
)
df_multi

In [ ]:
df_multi[("Temperature", "F")] = df_multi["Temperature"]["C"] * 9/5 + 32
df_multi

`stack()`: column --> row

In [ ]:
s = df_multi.stack()
s

In [ ]:
ss = s.stack()
ss

`unstack()`: row --> column

In [ ]:
ss.unstack()